In [18]:
import numpy as np
import pandas as pd
import scanpy as sc

import muon as mu
from muon import MuData
from muon import prot as pt

#for path in ['../../datasets/Mouse_Thymus5/']:
for path in ['../datasets/10x_human_lymph_node_A1/','../datasets/10x_human_lymph_node_D1/']:

    adata_omics1 = sc.read_h5ad(path + 'adata_RNA.h5ad')
    adata_omics2 = sc.read_h5ad(path + 'adata_ADT.h5ad')
    
    adata_omics1.var_names_make_unique()
    adata_omics2.var_names_make_unique()
    
    
    sc.pp.filter_genes(adata_omics2, min_cells=10)
    sc.pp.highly_variable_genes(adata_omics1, flavor="seurat_v3", n_top_genes=3000)
    sc.pp.normalize_total(adata_omics1, target_sum=1e4)
    sc.pp.log1p(adata_omics1)
      
    #adata_omics1_high =  adata_omics1[:, adata_omics1.var['highly_variable']]
    sc.pp.pca(adata_omics1,n_comps=30)
      
    #Protein
    pt.pp.clr(adata_omics2)
    sc.pp.scale(adata_omics2)
    sc.pp.pca(adata_omics2,n_comps=30)
    
    
    mdata = MuData({'rna': adata_omics1, 'adt': adata_omics2})
    
    mu.tl.mofa(mdata,n_factors=30,use_float32=True,gpu_mode=True,gpu_device=1)
    
    rna=mdata["rna"]
    rna.obsm["X_mofa"]=mdata.obsm["X_mofa"]
    # import os

    # os.environ['R_HOME'] = '/home/ws6tg/anaconda3/envs/EnvR43/lib/R/'
    # from SpatialGlue.utils import clustering
    # clustering(rna, key="X_mofa", add_key=f"11", n_clusters=5, method="mclust", use_pca=True)
    
    result_path=path.replace("datasets","results")
    rna.write_h5ad(result_path+'/adata_mofa.h5ad')
    print("Save!"+result_path+'/adata_mofa.h5ad')


        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        
use_float32 set to True: replacing float64 arrays by float32 arrays to speed up computations...

Loaded view='rna' group='group1' with N=3484 samples and D=18085 features...
Loaded view='adt' group='group1' with N=3484 samples and D=31 features...



Model options:
- Automatic Relevance Determination prior on the factors: True
- Automatic Relevance Determination prior on the wei